In [9]:
from poincare import Parameter, System, Variable, assign, initial
from poincare.compile import (
    Node,
    assignment,
    build_first_order_symbolic_ode,
    substitute,
)


class Loop(System):
    x: Variable = initial(default=0)
    y: Variable = initial(default=0)
    rate: Parameter = assign(default=1)
    r_x = x.derive() << rate * x
    r_y = y.derive() << rate * y


class Main(System):
    x: Variable = initial(default=0)
    rate: Parameter = assign(default=1)
    r = x.derive() << rate * x
    loop = Loop(y=x)

In [10]:
from typing import Iterator


def split_main_and_loop(
    iterable: Iterator[Node],
    main: type[System],
    loop: System,
):
    main_nodes = []
    loop_nodes = []
    for n in iterable:
        parent = n.parent
        while True:
            if parent is loop:
                loop_nodes.append(n)
                break
            elif parent is main:
                main_nodes.append(n)
                break
            elif parent is None:
                raise ValueError
            else:
                parent = parent.parent
    return main_nodes, loop_nodes


compiled = build_first_order_symbolic_ode(Main)
variables_main, variables_loop = split_main_and_loop(
    compiled.variables, Main, Main.loop
)
parameters_main, parameters_loop = split_main_and_loop(
    compiled.parameters, Main, Main.loop
)
func_main, func_loop = split_main_and_loop(compiled.func.keys(), Main, Main.loop)
output_main, output_loop = split_main_and_loop(
    compiled.output.values(), Main, Main.loop
)

In [11]:
from symbolite.abstract import scalar, vector

t = scalar.Scalar("t")
y = vector.Vector("y")
p = vector.Vector("p")
y_offset = scalar.Scalar("y_offset")
p_offset = scalar.Scalar("p_offset")
loop_i = scalar.Scalar("loop_i")
y_step = scalar.Scalar("y_step")
p_step = scalar.Scalar("p_step")
mapping = {
    compiled.independent: t,
}
for i, v in enumerate(variables_main):
    mapping[v] = y[i]
for i, v in enumerate(parameters_main):
    mapping[v] = p[i]
for i, v in enumerate(variables_loop):
    mapping[v] = y[y_offset + loop_i * y_step + i]
for i, v in enumerate(parameters_loop):
    mapping[v] = p[p_offset + loop_i * p_step + i]

In [12]:
for k in func_main:
    print(k, compiled.func[k])
    print(k, substitute(compiled.func[k], mapping))

x rate * x + loop.rate * x
x p[0] * y[0] + p[(p_offset + loop_i * p_step + 0)] * y[0]


In [21]:
func_lines = ["def ode_step(t, y, p, ydot):"]
for k in func_main:
    eq = substitute(compiled.func[k], mapping)
    line = assignment("ydot", variables_main.index(k), str(eq))
    func_lines.append(line)
func_lines.append("return ydot")
func = "\n    ".join(func_lines)
print(func)

def ode_step(t, y, p, ydot):
    ydot[0] = 1.0 * p[0] + 1.0 * p[(p_offset + loop_i * p_step + 1)]
    return ydot


In [21]:
y_offset, y_step = 1, 1
p_offset, p_step = 1, 2
indent = 4 * " "
loop = "\n".join(
    [
        f"N_loops = (y.size - {y_offset}) // {y_step}",
        "for loop_num in range(N_loops):",
        f"{indent}y_offset = loop_num * {y_step} + {y_offset}",
        f"{indent}p_offset = loop_num * {p_step} + {p_offset}",
    ]
)

print(loop)

N_loops = (y.size - 1) // 1
for loop_num in range(N_loops):
    y_offset = loop_num * 1 + 1
    p_offset = loop_num * 2 + 1
